In [1]:
# Rodrigo Caye Daudt
# rodrigo.cayedaudt@geod.baug.ethz.ch
# 04/2021

%matplotlib inline
import numpy as np
from skimage import io
import matplotlib
import matplotlib.pyplot as plt
import os
from scipy.stats import multivariate_normal
from sklearn.svm import LinearSVC

import utils

%load_ext autoreload
%autoreload 2

if not os.path.exists('./outputs'):
    os.mkdir('./outputs')

In [5]:
import os
os.getcwd()

'/home/gsa/code/remote_sensing/Lab_2/L2W2'

In [2]:
# Read images and labels

# Sentinel band names
band_names = [
    'band2', 
    'band3', 
    'band4', 
    'band5', 
    'band6', 
    'band7', 
    'band8', 
    'band8a', 
    'band11', 
    'band12', 
]

# Band wavelengths in nm
wavelengths = [490, 560, 665, 705, 740, 783, 842, 865, 1610, 2190]

# Load images
img_dict = {}
for bn in band_names:
    img_dict[bn] = io.imread(f'../raw_data/sentinel2_{bn}.tif') / (2 ** 16)


# Save non-georeferenced RGB for evaluating results
rgb = np.zeros((img_dict[band_names[0]].shape[0], img_dict[band_names[0]].shape[1], 3))
for i, band in enumerate(['band4', 'band3', 'band2']):
    rgb[:,:,i] = img_dict[band] * (2 ** 5)
io.imsave(f'outputs/00-rgb.png', np.clip(rgb, 0, 1))


# Class labels
labels = io.imread('labels.tif')

# Class masks
classes_dict = {
    'forest': 1,
    'water': 2,
    'clouds': 3,
    'fields (green)': 4,
    'fields (brown)': 5,
    'cities': 6,
    'snow': 7,
    'rock': 8,
}

# Extract class masks from labels
masks_dict = {}
min_np = np.inf # minimum number of pixels in a class
for class_name in classes_dict.keys():
    masks_dict[class_name] = labels == classes_dict[class_name]
    print(f'Number of pixels in class "{class_name}": {masks_dict[class_name].sum()}')
    min_np = min(min_np, masks_dict[class_name].sum())

if min_np < 100:
    print('WARNING: at least one of the classes contain very few examples (<100 pixels).')


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


Number of pixels in class "forest": 11722
Number of pixels in class "water": 148204
Number of pixels in class "clouds": 83733
Number of pixels in class "fields (green)": 5188
Number of pixels in class "fields (brown)": 1130
Number of pixels in class "cities": 47144
Number of pixels in class "snow": 104004
Number of pixels in class "rock": 82132


# Bayes Rule Classifier

In [3]:
# Calculate mean per band and per class

mean_dict = {}
cov_dict = {}
data_dict = {}
for class_name in classes_dict.keys():
    # Mean
    spectral_profile = []
    for band in band_names:
        spectral_profile.append(np.mean(img_dict[band][masks_dict[class_name]]))
    mean_dict[class_name] = spectral_profile

    # Assemble data for all bands in pixels of the current class in matrix form
    x = np.zeros((len(band_names), masks_dict[class_name].sum())) # axis 0: class index | axis 1: instance index
    for i, band in enumerate(band_names):
#         x[i] = 'FILL_IN_YOUR_CODE_HERE' # subtracting mean is not necessary if using np.cov
        x[i] = img_dict[band][masks_dict[class_name]]# subtracting mean is not necessary if using np.cov
    data_dict[class_name] = x

    # Calculate covariance
    cov_dict[class_name] = np.cov(x)
    
    


In [8]:
s0, s1 = img_dict[band].shape
s0 * s1, rgb.shape, masks_dict[class_name].sum()
img_dict[band][masks_dict[class_name]].shape

(82132,)

In [4]:
# x = np.zeros((len(band_names), masks_dict[class_name].sum()))
x.shape
class_name
np.sum(x == 0)
np.sum(masks_dict[class_name])
data_dict['rock'].shape


(10, 82132)

In [7]:
# Calculate the pixel probabilities for each class

# Stack all bands into a single "image"
stack = np.zeros((img_dict[band_names[0]].shape[0], img_dict[band_names[0]].shape[1], len(band_names)))
for i, band in enumerate(band_names):
    stack[:,:,i] = img_dict[band]

# Form a matrix where each row represent a pixel
stack_vec = stack.reshape(-1,len(band_names)) # matrix form for parallel processing of pixels


# Get per pixel class unnormalized probabilities
probs = np.zeros((img_dict[band_names[0]].shape[0], img_dict[band_names[0]].shape[1], len(classes_dict)))
# probs_scipy = np.zeros((img_dict[band_names[0]].shape[0], img_dict[band_names[0]].shape[1], len(classes_dict)))
for i, class_name in enumerate(classes_dict.keys()):
    print(f'\nProcessing class {class_name}...')

    ################################################################################
    # Fast version using scipy
    # IMPORTANT: the scipy approach here will not be evaluated, it should only be used to check your results if you want.

#     probs_vec = multivariate_normal.pdf(stack_vec, mean=mean_dict[class_name], cov=cov_dict[class_name])
#     probs_scipy[:,:,i] = np.reshape(probs_vec, stack.shape[:2])

    ################################################################################
    # Self coded version

    # Go into utils.py and fill in the missing code
    # Note: this should take around 10 minutes to process all classes.
    probs[:,:,i] = utils.pdf(stack, mean_dict[class_name], cov_dict[class_name])

# Normalize probabilities per pixel
# Be careful not to divide by 0
# probs = 'FILL_IN_YOUR_CODE_HERE'

s = np.sum(probs, axis=-1) + 1e-10
for i in range(8):
    probs[:, :, i] /= s



Processing class forest...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2861/2861 [00:50<00:00, 56.85it/s]



Processing class water...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2861/2861 [00:53<00:00, 53.85it/s]



Processing class clouds...


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2861/2861 [00:49<00:00, 57.24it/s]



Processing class snow...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2861/2861 [00:51<00:00, 55.46it/s]



Processing class rock...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2861/2861 [00:50<00:00, 56.56it/s]


In [20]:
np.sum(probs[0][0] / np.sum(probs[0][0]))
s = np.sum(probs, axis=-1)
for i in range(8):
    probs[:, :, i] /= s
probs

/tmp/ipykernel_15414/723749962.py:4: RuntimeWarning: invalid value encountered in true_divide
  probs[:, :, i] /= s


array([[[7.91096441e-167, 0.00000000e+000, 1.67218124e-001, ...,
         8.32781439e-001, 0.00000000e+000, 4.36716756e-007],
        [2.43500948e-099, 0.00000000e+000, 6.25400431e-002, ...,
         9.37459739e-001, 0.00000000e+000, 2.18102402e-007],
        [5.81788488e-073, 0.00000000e+000, 8.59344652e-002, ...,
         9.14064300e-001, 0.00000000e+000, 1.23511389e-006],
        ...,
        [6.64112777e-001, 0.00000000e+000, 4.60031286e-005, ...,
         3.32603222e-001, 0.00000000e+000, 6.37670299e-009],
        [1.23191590e-004, 0.00000000e+000, 7.49004938e-006, ...,
         2.41478781e-002, 0.00000000e+000, 3.70625001e-008],
        [1.11013964e-009, 0.00000000e+000, 2.39279122e-008, ...,
         1.79221681e-004, 0.00000000e+000, 9.26439055e-012]],

       [[7.14654347e-122, 0.00000000e+000, 1.71053530e-004, ...,
         9.99828946e-001, 0.00000000e+000, 1.02114398e-012],
        [1.97992014e-061, 0.00000000e+000, 1.13077965e-006, ...,
         9.99998869e-001, 0.00000000e+

In [47]:
# np.argmax(probs[:, :, ])
n = 8
1. - (np.max(probs, axis=-1) - np.sum(probs, axis=-1) / n) / (1 - 1./n)
# np.sum(probs, axis=-1).shape
# class_name
np.sum(probs, axis=-1)

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [21]:
for i, class_name in enumerate(classes_dict.keys()):
    io.imsave(f'outputs/01-0{i+1}-{class_name}.png', probs[:,:,i])

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [42]:
# Harden: find the class (index) with the highest likelihood at each pixel
# Note: indices are 0-7 and classes are 1-8. You  should fix this before saving the result
max_likelihood = np.argmax(probs, axis=-1) + 1
io.imsave(f'outputs/02-max_likelihood.png', max_likelihood.astype('uint8'))

n = 8

# Uncertainty map: calculate the uncertainty at each pixel using the provided formula
uncertainty_map = 1. - (np.max(probs, axis=-1) - np.sum(probs, axis=-1) / n) / (1 - 1./n)
io.imsave(f'outputs/03-uncertainty_map.png', uncertainty_map)

/tmp/ipykernel_15414/2308104301.py:4: UserWarning: outputs/02-max_likelihood.png is a low contrast image
  io.imsave(f'outputs/02-max_likelihood.png', max_likelihood.astype('uint8'))
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


# Support Vector Machine (SVM)

In [71]:

# Organize inputs and outputs

# Only labelled pixels should be used for training the SVM
X = np.zeros((0,len(band_names))) # each row represents a pixel, each column contains that pixel's brightness in each channel
y = np.zeros(0) # class number of each pixel
for i, class_name in enumerate(classes_dict.keys()):
    # Tip: variable stack already contains the stacked images
    # Concatenate to X all the pixels in class class_name
    X = np.concatenate((X, stack[masks_dict[class_name]]))

    # Concatenate to y the class index i the appropriate number of times (the same number of rows that were added to X)
    y = np.concatenate((y, (masks_dict[class_name][masks_dict[class_name]] * i)))

# Create SVM object using LinearSVC
svm = LinearSVC()

# Fit SVM to data using svm.fit
svm = svm.fit(X, y)

# Predict labels for all pixels using svm.predict and reshape to recopver image structure
# Data is already organised in stack_vec
# Predictions are made in a vectorized form, and need to be reshaped into image form
# Note: indices are 0-7 and classes are 1-8. You  should fix this before saving the result
# Use svm.predict to predict the class of each row in a matrix
svm_out = svm.predict(stack_vec) + 1
svm_out = np.reshape(svm_out, ((2861, 2870)))
io.imsave(f'outputs/04-svm.png', svm_out.astype('uint8'))

/tmp/ipykernel_15414/2208766387.py:27: UserWarning: outputs/04-svm.png is a low contrast image
  io.imsave(f'outputs/04-svm.png', svm_out.astype('uint8'))


In [70]:
classes_dict


{'forest': 1,
 'water': 2,
 'clouds': 3,
 'fields (green)': 4,
 'fields (brown)': 5,
 'cities': 6,
 'snow': 7,
 'rock': 8}